In [1]:
using Printf
using Statistics

using Flux
using DifferentialEquations
using DiffEqFlux

using JLD2
using Plots

┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/Flux/QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/DifferentialEquations/UQdwS.ji for DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/DiffEqFlux/BdO4p.ji for DiffEqFlux [aae7a2af-3d4f-5e19-a356-7da93b79d9d0]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/JLD2/O1EyT.ji for JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


In [2]:
file = jldopen("/home/gridsan/aramadhan/ocean_convection_profiles.jld2");

In [3]:
Is = keys(file["timeseries/t"])

Nz = file["grid/Nz"]
Nt = length(Is)

t = zeros(Nt)
T = T_data = zeros(Nt, Nz)

for (i, I) in enumerate(Is)
    t[i] = file["timeseries/t/$I"]
    T[i, :] = file["timeseries/T/$I"][1, 1, 2:Nz+1]
end

In [4]:
z = file["grid/zC"]

anim = @gif for n=1:Nt
    t_str = @sprintf("%.2f", t[n] / 86400)
    plot(T[n, :], z, linewidth=2,
         xlim=(19, 20), ylim=(-100, 0), label="",
         xlabel="Temperature (C)", ylabel="Depth (z)",
         title="Free convection: $t_str days", show=false)
end

display(anim)

"$(n)/$(Nt)" = "1/1153"
"$(n)/$(Nt)" = "2/1153"
"$(n)/$(Nt)" = "3/1153"
"$(n)/$(Nt)" = "4/1153"
"$(n)/$(Nt)" = "5/1153"
"$(n)/$(Nt)" = "6/1153"
"$(n)/$(Nt)" = "7/1153"
"$(n)/$(Nt)" = "8/1153"
"$(n)/$(Nt)" = "9/1153"
"$(n)/$(Nt)" = "10/1153"
"$(n)/$(Nt)" = "11/1153"
"$(n)/$(Nt)" = "12/1153"
"$(n)/$(Nt)" = "13/1153"
"$(n)/$(Nt)" = "14/1153"
"$(n)/$(Nt)" = "15/1153"
"$(n)/$(Nt)" = "16/1153"
"$(n)/$(Nt)" = "17/1153"
"$(n)/$(Nt)" = "18/1153"
"$(n)/$(Nt)" = "19/1153"
"$(n)/$(Nt)" = "20/1153"
"$(n)/$(Nt)" = "21/1153"
"$(n)/$(Nt)" = "22/1153"
"$(n)/$(Nt)" = "23/1153"
"$(n)/$(Nt)" = "24/1153"
"$(n)/$(Nt)" = "25/1153"
"$(n)/$(Nt)" = "26/1153"
"$(n)/$(Nt)" = "27/1153"
"$(n)/$(Nt)" = "28/1153"
"$(n)/$(Nt)" = "29/1153"
"$(n)/$(Nt)" = "30/1153"
"$(n)/$(Nt)" = "31/1153"
"$(n)/$(Nt)" = "32/1153"
"$(n)/$(Nt)" = "33/1153"
"$(n)/$(Nt)" = "34/1153"
"$(n)/$(Nt)" = "35/1153"
"$(n)/$(Nt)" = "36/1153"
"$(n)/$(Nt)" = "37/1153"
"$(n)/$(Nt)" = "38/1153"
"$(n)/$(Nt)" = "39/1153"
"$(n)/$(Nt)" = "40/1153"
"$(n)/$(N

Plots.AnimatedGif("/home/gridsan/aramadhan/6S898-climate-parameterization/notebooks/tmp.gif")

┌ Info: Saved animation to 
│   fn = /home/gridsan/aramadhan/6S898-climate-parameterization/notebooks/tmp.gif
└ @ Plots /home/gridsan/aramadhan/.julia/packages/Plots/Iuc9S/src/animation.jl:95


In [5]:
function coarse_grain(data, resolution)
    @assert length(data) % resolution == 0
    s = length(data) / resolution
    
    data_cs = zeros(resolution)
    for i in 1:resolution
        t = data[Int((i-1)*s+1):Int(i*s)]
        data_cs[i] = mean(t)
    end
    
    return data_cs
end

coarse_grain (generic function with 1 method)

In [6]:
coarse_resolution = cr = 32
T_cs = zeros(Nt, coarse_resolution)
for n=1:Nt
    T_cs[n, :] = coarse_grain(T[n, :], coarse_resolution)
end

T_cs = transpose(T_cs) |> Array;

In [20]:
dTdt_NN = Chain(Dense(cr, 2cr, tanh),
             Dense(2cr, cr))

ps = Flux.params(dTdt_NN)

# n_subset = round(Int, Nt/2)
# t_subset = t[6:n_subset] ./ 86400
# tspan = (t_subset[1], t_subset[end])
# neural_pde_prediction = T₀ -> neural_ode(dTdt_NN, T₀, tspan, Tsit5(), saveat=t_subset, reltol=1e-7, abstol=1e-9)

T₀ = T_cs[:, 1]
n_train = round(Int, Nt/2)
t_train = t[1:n_train] ./ 86400
tspan_train = (t_train[1], t_train[end])
neural_pde_prediction = T₀ -> neural_ode(dTdt_NN, T₀, tspan_train, Tsit5(), saveat=t_train, reltol=1e-7, abstol=1e-9)

#11 (generic function with 1 method)

In [21]:
data = Iterators.repeated((), 1000)
opt = ADAM(0.1)

loss_function() = sum(abs2, T_cs[:, 1:n_train] .- neural_pde_prediction(T₀))

loss_function (generic function with 1 method)

In [28]:
# Callback function to observe training.
cb = function ()
    loss = loss_function()
    println("loss = $loss")
    loss < 5 && Flux.stop()
end

#17 (generic function with 1 method)

In [29]:
Flux.train!(loss_function, ps, data, opt, cb = cb)

loss = 10.485636569307056 (tracked)
loss = 8.864905810938026 (tracked)
loss = 8.664184534364564 (tracked)
loss = 9.127431005083967 (tracked)
loss = 7.876071176336581 (tracked)
loss = 8.002101379827124 (tracked)
loss = 7.992327143395162 (tracked)
loss = 7.38705594839318 (tracked)
loss = 7.42140482052862 (tracked)
loss = 7.051036816126372 (tracked)
loss = 7.266548998813976 (tracked)
loss = 6.654429136622775 (tracked)
loss = 6.788699457811538 (tracked)
loss = 6.719816465564717 (tracked)
loss = 6.463083541651773 (tracked)
loss = 6.3676519070605355 (tracked)
loss = 6.437877148319099 (tracked)
loss = 6.236392284068109 (tracked)
loss = 6.12848605410504 (tracked)
loss = 6.212566178417359 (tracked)
loss = 6.001539811830767 (tracked)
loss = 6.094892195043866 (tracked)
loss = 5.865534561663464 (tracked)
loss = 6.024647875041499 (tracked)
loss = 5.869584328960228 (tracked)
loss = 5.797501676718619 (tracked)
loss = 5.898517945586018 (tracked)
loss = 5.7802387434128555 (tracked)
loss = 5.73114025724

InterruptException: InterruptException:

In [30]:
tspan = (t[1], t[end]) ./ 86400
nn_pred = neural_ode(dTdt_NN, T₀, tspan, Tsit5(), saveat=t ./86400, reltol=1e-7, abstol=1e-9) |> Flux.data

z_cs = coarse_grain(z, cr)

anim = @gif for n=1:Nt
    t_str = @sprintf("%.2f", t[n] / 86400)
    plot(T_cs[:, n], z_cs, linewidth=2,
         xlim=(19, 20), ylim=(-100, 0), label="Data",
         xlabel="Temperature (C)", ylabel="Depth (z)",
         title="Free convection: $t_str days",
         legend=:bottomright, show=false)
    if n <= n_subset
        plot!(nn_pred[:, n], z_cs, linewidth=2, label="Neural ODE (train)", show=false)
    else
        plot!(nn_pred[:, n], z_cs, linewidth=2, linestyle=:dash, label="Neural ODE (test)", show=false)
    end
end

┌ Info: Saved animation to 
│   fn = /home/gridsan/aramadhan/6S898-climate-parameterization/notebooks/tmp.gif
└ @ Plots /home/gridsan/aramadhan/.julia/packages/Plots/Iuc9S/src/animation.jl:95


Plots.AnimatedGif("/home/gridsan/aramadhan/6S898-climate-parameterization/notebooks/tmp.gif")